# Krypton MAU

## Study the use of a MAU to remove baseline oscillation

In [ ]:
import datetime
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
from __future__ import print_function
import sys
import os
from glob import glob
from time import time

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import pandas as pd
import tables as tb
import numpy as np

In [ ]:
from invisible_cities.database import load_db
from invisible_cities.core.system_of_units_c import SystemOfUnits
import invisible_cities.sierpe.blr as blr
import invisible_cities.core.mpl_functions as mpl
import invisible_cities.core.wfm_functions as wfm
import invisible_cities.core.tbl_functions as tbl
import invisible_cities.core.peak_functions_c as cpf
import invisible_cities.core.pmaps_functions as pf
import invisible_cities.core.sensor_functions as sf
import invisible_cities.core.core_functions as cf

In [ ]:
from scipy import signal as SGN

In [ ]:
%load_ext Cython

### Access to data base and definition of units

In [ ]:
DataPMT = load_db.DataPMT(2980)
units = SystemOfUnits()
adc_to_pes = abs(DataPMT.adc_to_pes.values)
coeff_c = abs(DataPMT.coeff_c.values)
coeff_blr = abs(DataPMT.coeff_blr.values)
DataSiPM = load_db.DataSiPM(2980)
adc_to_pes_sipm = DataSiPM.adc_to_pes.values
xs = DataSiPM.X.values
ys = DataSiPM.Y.values

### Data files

In [ ]:
RWF_file = os.environ['IC_DATA']  + '/LSC/run_2980.gdc1next.999.h5'

In [ ]:
h5rwf = tb.open_file(RWF_file,'r')

In [ ]:
pmtrwf, pmtblr, sipmrwf = tbl.get_vectors(h5rwf)

In [ ]:
NEVT, NPMT, PMTWL = pmtrwf.shape
NEVT, NSIPM, SIPMWL = sipmrwf.shape
print("""
        Number of events in file = {}
        Number of PMTs = {}
        PMTWL = {}
        Number of SiPMs = {}
        SiPMWL = {}
      """.format(NEVT, NPMT, PMTWL,NSIPM, SIPMWL))

### MC files

In [ ]:
RWF_MC = os.environ['IC_DATA']  + '/NEXT_v0_8_09/dst_NEXT_v0_08_09_Kr_ACTIVE_9_0_7bar_RWF_10000.root.h5'

In [ ]:
h5mc = tb.open_file(RWF_MC,'r')

In [ ]:
pmtrwfmc, pmtblrmc, sipmrwfmc = tbl.get_vectors(h5mc)

In [ ]:
NEVT, NPMT, PMTWL = pmtrwfmc.shape
NEVT, NSIPM, SIPMWL = sipmrwfmc.shape
print("""
        Number of events in file = {}
        Number of PMTs = {}
        PMTWL = {}
        Number of SiPMs = {}
        SiPMWL = {}
      """.format(NEVT, NPMT, PMTWL,NSIPM, SIPMWL))

### DATA

### Detailed analysis of event 0

In [ ]:
event = 0
RWF = pmtrwf[event]

In [ ]:
tstep = 25
PMTWL = 48000
signal_t = np.arange(0., PMTWL * tstep, tstep)/units.mus

In [ ]:
def ei(t):
    """returns the index in energy corresponding to time t"""
    tstep = 25
    PMTWL = 48000
    signal_t = np.arange(0., PMTWL * tstep, tstep)/units.mus
    return cf.loc_elem_1d(signal_t, t)

In [ ]:
wfm.plot_pmt_waveforms(RWF, zoom=False, window_size=2000)

#### A double event!

### Deconvolution 

In [ ]:
event=0
t0 = time()
CWF = blr.deconv_pmt(RWF, coeff_c, coeff_blr, n_baseline=38000, thr_trigger=5)
t1 = time()        
dt = t1 - t0

print("run over  one event  in {} s".format(dt))

In [ ]:
mpl.plot_pmt_signals_vs_time_mus(CWF,
                                 t_min=0,
                                 t_max=1200,
                                 signal_min=-10,
                                 signal_max=250)

In [ ]:
mpl.plot_pmt_signals_vs_time_mus(CWF, 
                                 t_min=0,
                                 t_max=590,
                                 signal_min=-10,
                                 signal_max=10)

In [ ]:
mpl.plot_pmt_signals_vs_time_mus(CWF, 
                                 t_min=590,
                                 t_max=620,
                                 signal_min=-5,
                                 signal_max=200)

In [ ]:
mpl.plot_pmt_signals_vs_time_mus(CWF, 
                                 t_min=610,
                                 t_max=820,
                                 signal_min=-10,
                                 signal_max=10)

#### Using a MAU to eliminate sinusoidal noise. 

This function is a copy of the function found in peak_functions.py, but returns the MAU, for ilustration purposes

In [ ]:
def calibrated_pmt_sum_py(CWF, adc_to_pes, n_MAU=200, thr_MAU=5):
    """Compute the ZS calibrated sum of the PMTs
    after correcting the baseline with a MAU to suppress low frequency noise.
    input:
    CWF         : Corrected waveform (passed by BLR)
    adc_to_pes  : a vector with calibration constants
    n_MAU       : length of the MAU window
    thr_MAU     : treshold above MAU to select sample

    NB: This function is used mainly for testing purposes. It is
    programmed "c-style", which is not necesarily optimal in python,
    but follows the same logic that the corresponding cython function
    (in peak_functions_c), which runs faster and should be used
    instead of this one for nornal calculations.
    """

    NPMT = CWF.shape[0]
    NWF  = CWF.shape[1]
    MAU  = np.array(np.ones(n_MAU), dtype=np.double) * (1 / n_MAU)

    pmt_thr = np.zeros((NPMT, NWF), dtype=np.double)
    csum    = np.zeros(       NWF,  dtype=np.double)
    csum_mau    = np.zeros(       NWF,  dtype=np.double)
    MAU_pmt = np.zeros(       NWF,  dtype=np.double)

    MAUL=[]
    for j in range(NPMT):
        # MAU for each of the PMTs, following the waveform
        MAU_pmt = SGN.lfilter(MAU, 1, CWF[j,:])
        MAUL.append(MAU_pmt)
        csum += CWF[j] * 1 / adc_to_pes[j]
        for k in range(NWF):
            if CWF[j,k] >= MAU_pmt[k] + thr_MAU: # >= not >. Found testing
                pmt_thr[j,k] = CWF[j,k]
        csum_mau += pmt_thr[j] * 1 / adc_to_pes[j]
    return csum, csum_mau, np.array(MAUL)


In [ ]:
%%cython
from scipy import signal as SGN
cimport numpy as np
import  numpy as np
cpdef calibrated_pmt_sum(double [:, :] CWF,
                         double [:] adc_to_pes,
                         int      n_MAU = 200,
                         double thr_MAU =   5):
    """
    Computes the ZS calibrated sum of the PMTs
    after correcting the baseline with a MAU to suppress low frequency noise.
    input:
    CWF:    Corrected waveform (passed by BLR)
    adc_to_pes: a vector with calibration constants
    n_MAU:  length of the MAU window
    thr_MAU: treshold above MAU to select sample

    """

    cdef int j, k
    cdef int NPMT = CWF.shape[0]
    cdef int NWF  = CWF.shape[1]
    cdef double [:] MAU = np.array(np.ones(n_MAU),
                                   dtype = np.double) * (1 / n_MAU)


    # CWF if above MAU threshold
    cdef double [:, :] pmt_thr = np.zeros((NPMT,NWF), dtype=np.double)
    cdef double [:]    csum = np.zeros(         NWF , dtype=np.double)
    cdef double [:]    csum_mau = np.zeros(     NWF , dtype=np.double)
    cdef double [:]    MAU_pmt = np.zeros(      NWF , dtype=np.double)

    for j in range(NPMT):
        # MAU for each of the PMTs, following the waveform
        MAU_pmt = SGN.lfilter(MAU, 1, CWF[j,:])

        for k in range(NWF):
            if CWF[j,k] >= MAU_pmt[k] + thr_MAU: # >= not >: found testing!
                pmt_thr[j,k] = CWF[j,k]

    for j in range(NPMT):
        for k in range(NWF):
            csum_mau[k] += pmt_thr[j, k] * 1 / adc_to_pes[j]
            csum[k] += CWF[j, k] * 1 / adc_to_pes[j]
    return np.asarray(csum), np.asarray(csum_mau)

In [ ]:
event=0
t0 = time()
csum, csum_mau, MAU = calibrated_pmt_sum_py(CWF, adc_to_pes, n_MAU=400, thr_MAU=5) 
t1 = time()        
dt = t1 - t0

print("run over  one event  in {} s".format(dt))

In [ ]:
mpl.plot_pmt_signals_vs_time_mus(MAU, 
                                 t_min=0,
                                 t_max=590,
                                 signal_min=-2,
                                 signal_max=2)

In [ ]:
event=0
t0 = time()
csum, csum_mau = calibrated_pmt_sum(CWF, adc_to_pes, n_MAU=400, thr_MAU=5) 
t1 = time()        
dt = t1 - t0
print("run over  one event  in {} s".format(dt))

In [ ]:
tmin=0
tmax=1200
mpl.plot_signal_vs_time_mus(csum, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=100)
mpl.plot_signal_vs_time_mus(csum_mau, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=100)

In [ ]:
tmin=0
tmax=590
mpl.plot_signal_vs_time_mus(csum, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=2)

In [ ]:
tmin=0
tmax=590
mpl.plot_signal_vs_time_mus(csum_mau, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=2)

#### MAU eliminates sinusoidal noise and leaves only "S1" like signals.

In [ ]:
tmin=590
tmax=610
mpl.plot_signal_vs_time_mus(csum, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=60)
mpl.plot_signal_vs_time_mus(csum_mau, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=60)

#### MAU introduces a bias in S2. The search can be done with csum_mau but one should take the area from csum

In [ ]:
np.sum(csum[ei(tmin):ei(tmax)])

In [ ]:
np.sum(csum_mau[ei(tmin):ei(tmax)])

In [ ]:
tmin=610
tmax=820
mpl.plot_signal_vs_time_mus(csum, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=2)

In [ ]:
mpl.plot_signal_vs_time_mus(csum_mau, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=2)

In [ ]:
tmin=820
tmax=825
mpl.plot_signal_vs_time_mus(csum, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=100)
mpl.plot_signal_vs_time_mus(csum_mau, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=100)

In [ ]:
np.sum(csum[ei(tmin):ei(tmax)])

In [ ]:
np.sum(csum_mau[ei(tmin):ei(tmax)])

In [ ]:
tmin=825
tmax=1200
mpl.plot_signal_vs_time_mus(csum, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=2)

In [ ]:
tmin=825
tmax=1200
mpl.plot_signal_vs_time_mus(csum_mau, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=2)

### MC

In [ ]:
event = 0
RWF = pmtrwfmc[event]

In [ ]:
event=0
t0 = time()
CWF = blr.deconv_pmt(RWF, coeff_c, coeff_blr, n_baseline=38000, thr_trigger=5)
t1 = time()        
dt = t1 - t0

print("run over  one event  in {} s".format(dt))

In [ ]:
mpl.plot_pmt_signals_vs_time_mus(CWF,
                                 t_min=0,
                                 t_max=1200,
                                 signal_min=-10,
                                 signal_max=250)

In [ ]:
mpl.plot_pmt_signals_vs_time_mus(CWF, 
                                 t_min=0,
                                 t_max=400,
                                 signal_min=-10,
                                 signal_max=10)

In [ ]:
mpl.plot_pmt_signals_vs_time_mus(CWF, 
                                 t_min=400,
                                 t_max=500,
                                 signal_min=-5,
                                 signal_max=200)

In [ ]:
mpl.plot_pmt_signals_vs_time_mus(CWF, 
                                 t_min=470,
                                 t_max=800,
                                 signal_min=-10,
                                 signal_max=10)

In [ ]:
event=0
t0 = time()
csum, csum_mau, MAU = calibrated_pmt_sum_py(CWF, adc_to_pes, n_MAU=400, thr_MAU=5) 
t1 = time()        
dt = t1 - t0

print("run over  one event  in {} s".format(dt))

In [ ]:
mpl.plot_pmt_signals_vs_time_mus(MAU, 
                                 t_min=0,
                                 t_max=400,
                                 signal_min=-2,
                                 signal_max=2)

In [ ]:
event=0
t0 = time()
csum, csum_mau = calibrated_pmt_sum(CWF, adc_to_pes, n_MAU=400, thr_MAU=5) 
t1 = time()        
dt = t1 - t0
print("run over  one event  in {} s".format(dt))

In [ ]:
tmin=0
tmax=400
mpl.plot_signal_vs_time_mus(csum, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=2)

In [ ]:
tmin=0
tmax=400
mpl.plot_signal_vs_time_mus(csum_mau, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=2)

In [ ]:
tmin=100
tmax=100.5
mpl.plot_signal_vs_time_mus(csum_mau, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=2)

In [ ]:
tmin=100
tmax=100.5
mpl.plot_signal_vs_time_mus(csum, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=2)

In [ ]:
np.sum(csum[ei(tmin):ei(tmax)])

In [ ]:
np.sum(csum_mau[ei(tmin):ei(tmax)])

In [ ]:
tmin=440
tmax=480
mpl.plot_signal_vs_time_mus(csum, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=60)
mpl.plot_signal_vs_time_mus(csum_mau, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=60)

In [ ]:
tmin=470
tmax=1200
mpl.plot_signal_vs_time_mus(csum, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=2)

In [ ]:
mpl.plot_signal_vs_time_mus(csum_mau, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=2)

### A MAU-based search

1. Pass MAU in the pre-S2 and post-S2 region
2. Identify S1 candidates
3. Search for S1 near the candidate region. 

### ZS sum

#### USING MAU

In [ ]:
event = 0
t0 = time()
wfzs_ene, wfzs_indx = cpf.wfzs(csum, threshold=0.5*units.pes)
wfzs_t = cpf.time_from_index(wfzs_indx)
wfzs_ene2, wfzs_indx2 = cpf.wfzs(csum_mau, threshold=0.1*units.pes)
wfzs_t2 = cpf.time_from_index(wfzs_indx2)
t1 = time()        
dt = t1 - t0

print("run  in {} s".format(dt))

In [ ]:
mpl.plot_signal(wfzs_t/units.mus, wfzs_ene, title="calibrated sum",
                signal_start=0, signal_end=1200, 
                ymax = 100, t_units='mus', units="pes")

In [ ]:
mpl.plot_signal(wfzs_t/units.mus, wfzs_ene, title="calibrated sum",
                signal_start=0, signal_end=400, 
                ymax = 10, t_units='mus', units="pes")

### MAU

In [ ]:
mpl.plot_signal(wfzs_t/units.mus, wfzs_ene, title="calibrated sum",
                signal_start=0, signal_end=1200, 
                ymax = 100, t_units='mus', units="pes")

In [ ]:
mpl.plot_signal(wfzs_t2/units.mus, wfzs_ene2, title="calibrated sum",
                signal_start=0, signal_end=400, 
                ymax = 10, t_units='mus', units="pes")

### S12

In [ ]:
S1 = cpf.find_S12(wfzs_ene, wfzs_indx, tmin=0,  tmax=110*units.mus, 
                 lmin=6, lmax=16, stride=4,
                 rebin=False)
S2 = cpf.find_S12(wfzs_ene, wfzs_indx, tmin=110*units.mus,  tmax=1200*units.mus, 
                 lmin=80, lmax=1000, stride=40,
                 rebin=True, rebin_stride=40)

In [ ]:
S1

In [ ]:
S2

In [ ]:
pf.scan_s12(S1)

In [ ]:
pf.scan_s12(S2)

### S12 MAU

In [ ]:
S1 = cpf.find_S12(wfzs_ene2, wfzs_indx2, tmin=0,  tmax=110*units.mus, 
                 lmin=6, lmax=16, stride=4,
                 rebin=False)
S2 = cpf.find_S12(wfzs_ene2, wfzs_indx2, tmin=110*units.mus,  tmax=1200*units.mus, 
                 lmin=80, lmax=1000, stride=40,
                 rebin=True, rebin_stride=40)

In [ ]:
pf.scan_s12(S1)

In [ ]:
pf.scan_s12(S2)

### MAU is not specially useful in the MC

#### Event 1

In [ ]:
event=1
RWF = pmtrwf[event]
CWF = blr.deconv_pmt(RWF, coeff_c, coeff_blr, n_baseline=38000, thr_trigger=5)
csum, csum_mau = calibrated_pmt_sum(CWF, adc_to_pes, n_MAU=100, thr_MAU=5) 

In [ ]:
tmin=0
tmax=590
mpl.plot_signal_vs_time_mus(csum, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=2)

In [ ]:
tmin=0
tmax=590
mpl.plot_signal_vs_time_mus(csum_mau, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=2)

In [ ]:
tmin=590
tmax=610
mpl.plot_signal_vs_time_mus(csum, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=60)
mpl.plot_signal_vs_time_mus(csum_mau, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=60)

In [ ]:
tmin=610
tmax=1200
mpl.plot_signal_vs_time_mus(csum, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=2)
mpl.plot_signal_vs_time_mus(csum_mau, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=2)

In [ ]:
wfzs_ene, wfzs_indx = cpf.wfzs(csum, threshold=0.5*units.pes)
wfzs_t = cpf.time_from_index(wfzs_indx)
S1 = cpf.find_S12(wfzs_ene, wfzs_indx, tmin=0,  tmax=590*units.mus, 
                 lmin=6, lmax=16, stride=4,
                 rebin=False)
S2 = cpf.find_S12(wfzs_ene, wfzs_indx, tmin=10*units.mus,  tmax=1190*units.mus, 
                 lmin=80, lmax=1000, stride=40,
                 rebin=True, rebin_stride=40)

In [ ]:
S1

In [ ]:
S2

In [ ]:
pf.scan_s12(S2)

#### SiPMs

In [ ]:
sipm = cpf.signal_sipm(sipmrwf[event], adc_to_pes_sipm, thr=5*units.pes, n_MAU=100)
SIPM = cpf.select_sipm(sipm)
S2Si = pf.sipm_s2_dict(SIPM, S2, thr=50*units.pes)

In [ ]:
S2Si

In [ ]:
pf.plot_s2si_map(S2Si)

In [ ]:
event=2
RWF = pmtrwf[event]
CWF = blr.deconv_pmt(RWF, coeff_c, coeff_blr, n_baseline=38000, thr_trigger=5)
csum, csum_mau = calibrated_pmt_sum(CWF, adc_to_pes, n_MAU=400, thr_MAU=10) 
wfzs_ene, wfzs_indx = cpf.wfzs(csum, threshold=0.5*units.pes)
wfzs_t = cpf.time_from_index(wfzs_indx)
S1 = cpf.find_S12(wfzs_ene, wfzs_indx, tmin=0,  tmax=590*units.mus, 
                 lmin=6, lmax=16, stride=4,
                 rebin=False)
S2 = cpf.find_S12(wfzs_ene, wfzs_indx, tmin=0*units.mus,  tmax=1100*units.mus, 
                 lmin=50, lmax=1000, stride=40,
                 rebin=True, rebin_stride=40)
sipm = cpf.signal_sipm(sipmrwf[event], adc_to_pes_sipm, thr=5*units.pes, n_MAU=100)
SIPM = cpf.select_sipm(sipm)
S2Si = pf.sipm_s2_dict(SIPM, S2, thr=50*units.pes)

In [ ]:
tmin=0
tmax=590
mpl.plot_signal_vs_time_mus(csum, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=2)

In [ ]:
tmin=0
tmax=590
mpl.plot_signal_vs_time_mus(csum_mau, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=2)

In [ ]:
tmin=590
tmax=640
mpl.plot_signal_vs_time_mus(csum, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=200)
mpl.plot_signal_vs_time_mus(csum_mau, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=200)

In [ ]:
tmin=620
tmax=1200
mpl.plot_signal_vs_time_mus(csum, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=2)

In [ ]:
mpl.plot_signal_vs_time_mus(csum_mau, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=2)

1. Before signal: flat tail at about 0.1 pes.
2. After signal: rises to about 1 pes, then decays. 

In [ ]:
S1

In [ ]:
pf.scan_s12(S1)

In [ ]:
pf.scan_s12(S2)

#### Not a krypton signal. 

In [ ]:
pf.plot_s2si_map(S2Si)

A clear track!

In [ ]:
event=3
RWF = pmtrwf[event]
CWF = blr.deconv_pmt(RWF, coeff_c, coeff_blr, n_baseline=38000, thr_trigger=5)
csum, csum_mau = calibrated_pmt_sum(CWF, adc_to_pes, n_MAU=400, thr_MAU=5) 
wfzs_ene, wfzs_indx = cpf.wfzs(csum, threshold=0.5*units.pes)
wfzs_t = cpf.time_from_index(wfzs_indx)
S1 = cpf.find_S12(wfzs_ene, wfzs_indx, tmin=0,  tmax=590*units.mus, 
                 lmin=6, lmax=16, stride=4,
                 rebin=False)
S2 = cpf.find_S12(wfzs_ene, wfzs_indx, tmin=0*units.mus,  tmax=1100*units.mus, 
                 lmin=50, lmax=1000, stride=40,
                 rebin=True, rebin_stride=40)
sipm = cpf.signal_sipm(sipmrwf[event], adc_to_pes_sipm, thr=5*units.pes, n_MAU=100)
SIPM = cpf.select_sipm(sipm)
S2Si = pf.sipm_s2_dict(SIPM, S2, thr=50*units.pes)

In [ ]:
tmin=0
tmax=590
mpl.plot_signal_vs_time_mus(csum, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=2)

In [ ]:
mpl.plot_signal_vs_time_mus(csum_mau, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=2)

In [ ]:
tmin=590
tmax=610
mpl.plot_signal_vs_time_mus(csum, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=60)
mpl.plot_signal_vs_time_mus(csum_mau, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=60)

In [ ]:
tmin=610
tmax=1200
mpl.plot_signal_vs_time_mus(csum, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=2)

In [ ]:
mpl.plot_signal_vs_time_mus(csum_mau, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=2)

In [ ]:
pf.scan_s12(S1)

In [ ]:
pf.scan_s12(S2)

In [ ]:
pf.plot_s2si_map(S2Si)